In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os
from datetime import datetime
import pytz
from pathlib import Path
import sklearn as sk
import tensorflow as tf
import torch
import torch.nn as nn
from torchvision.io import read_image
import time
import timeit
import random
import torchvision.models as models
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import sklearn.metrics
import csv
import timeit

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls 'drive/MyDrive/DS5220: Final Project/logs/mobilenet_v2'

20240425_192621_logs/  20240425_205041_logs/  20240425_212853_logs/  20240425_215918_logs/


In [ ]:
BASE_LOGS_PATH = 'drive/MyDrive/DS5220: Final Project/logs/mobilenet_v2'

In [ ]:
# https://pytorch.org/docs/stable/notes/mps.html
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
CIFAR100_ROOT_PATH='drive/MyDrive/DS5220: Final Project'

class Cifar100():
    def __init__(self,
                 calculate_mean_and_std = False):
        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

        self.BATCH_SIZE = 128
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)])

        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True,
                                transform=self.transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                train=False,
                                transform=self.transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                               num_workers=0,
                               batch_size=self.BATCH_SIZE,
                               shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                              num_workers=0,
                              batch_size=self.BATCH_SIZE,
                              shuffle=False)

    def calculate_mean_and_std(self):
        train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True)
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        _mean = np.mean(x, axis=(0, 1))/255
        _std = np.std(x, axis=(0, 1))/255
        _mean = _mean.tolist()
        _std = _std.tolist()
        return _mean, _std

class Cifar100WithAugmentation():
    def __init__(self, calculate_mean_and_std=False):
        CIFAR100_ROOT_PATH = 'drive/MyDrive/DS5220: Final Project'
        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)
        self.BATCH_SIZE = 128
        self.train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.eval_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                               download=True,
                                               train=True,
                                               transform=self.train_transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                              train=False,
                                              transform=self.eval_transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                                            num_workers=4,
                                            batch_size=self.BATCH_SIZE,
                                            shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                                           num_workers=4,
                                           batch_size=self.BATCH_SIZE,
                                           shuffle=False)

    def calculate_mean_and_std(self):
        train_dataset = CIFAR100(root='drive/MyDrive/DS5220: Final Project',
                                          download=True,
                                          train=True)
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        mean = np.mean(x, axis=(0, 1)) / 255
        std = np.std(x, axis=(0, 1)) / 255
        return mean.tolist(), std.tolist()

cf100 = Cifar100(calculate_mean_and_std=True)
cf100_w_aug = Cifar100WithAugmentation(calculate_mean_and_std=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# mobilenetv2 model config
config = {
    'dataloader' : cf100_w_aug,
    'model': models.mobilenet_v2(),
    'epochs': 70,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time': None,
    'drop-out': False
}

model = config['model']
model.to(device)
EPOCHS = config['epochs']
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

#  log directory
if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")
else:
    print(f"Directory '{config['log_dir']}' already exists")

metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    training_start_time = timeit.default_timer()
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []

        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                #  top-1 accuracy
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                #  top-5 accuracy
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)

                eval_accuracies.append(top1_accuracy)
        # write training  to csv
        end_time = timeit.default_timer()
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")
    training_end_time = timeit.default_timer()
    training_duration = training_end_time - training_start_time
    config['run_time'] = training_duration

config_path = os.path.join(config['log_dir'], 'config.txt')
# output config file to directory
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')

torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))


Directory 'drive/MyDrive/DS5220: Final Project/logs/mobilenet_v2/20240426_193309_logs' created


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/70
Train Loss: 4.3493, Train Accuracy: 5.83%
Eval Loss: 3.8677, Eval Accuracy: 9.87%, Top-1 Accuracy: 9.87%, Top-5 Accuracy: 30.67%
Epoch 2/70
Train Loss: 3.7674, Train Accuracy: 11.53%
Eval Loss: 3.5640, Eval Accuracy: 14.89%, Top-1 Accuracy: 14.89%, Top-5 Accuracy: 40.86%
Epoch 3/70
Train Loss: 3.5610, Train Accuracy: 14.64%
Eval Loss: 3.4469, Eval Accuracy: 16.61%, Top-1 Accuracy: 16.61%, Top-5 Accuracy: 43.37%
Epoch 4/70
Train Loss: 3.4098, Train Accuracy: 17.43%
Eval Loss: 3.2627, Eval Accuracy: 19.77%, Top-1 Accuracy: 19.77%, Top-5 Accuracy: 48.30%
Epoch 5/70
Train Loss: 3.2811, Train Accuracy: 19.54%
Eval Loss: 3.1760, Eval Accuracy: 21.62%, Top-1 Accuracy: 21.62%, Top-5 Accuracy: 50.68%
Epoch 6/70
Train Loss: 3.1672, Train Accuracy: 21.89%
Eval Loss: 3.1531, Eval Accuracy: 22.20%, Top-1 Accuracy: 22.20%, Top-5 Accuracy: 51.93%
Epoch 7/70
Train Loss: 3.0580, Train Accuracy: 23.89%
Eval Loss: 2.9803, Eval Accuracy: 26.17%, Top-1 Accuracy: 26.17%, Top-5 Accuracy: 55.46%
Ep

In [ ]:
# custom mobilenetv2 model config wtih additional drop
class CustomMobileNetV2(nn.Module):
    def __init__(self, num_classes=100, dropout_rate=0.5):
        super(CustomMobileNetV2, self).__init__()
        self.mobilenet_v2 = models.mobilenet_v2(pretrained=False)
        self.mobilenet_v2.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.mobilenet_v2.last_channel, num_classes)
        )
    def forward(self, x):
        return self.mobilenet_v2(x)
config = {
    'dataloader' : cf100_w_aug,
    'model': CustomMobileNetV2(),
    'epochs': 70,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time' : None
}
model = config['model']
model.to(config['device'])
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")

metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []
        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(config['device']), labels.to(config['device'])
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(config['device']), labels.to(config['device'])
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)
                eval_accuracies.append(top1_accuracy)

        end_time = timeit.default_timer()
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")

torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))

Directory 'drive/MyDrive/DS5220: Final Project/logs/mobilenet_v2/20240426_201747_logs' created


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/70
Train Loss: 4.3228, Train Accuracy: 4.95%
Eval Loss: 3.9498, Eval Accuracy: 9.09%, Top-1 Accuracy: 9.09%, Top-5 Accuracy: 28.61%
Epoch 2/70
Train Loss: 3.9252, Train Accuracy: 9.47%
Eval Loss: 3.6308, Eval Accuracy: 14.43%, Top-1 Accuracy: 14.43%, Top-5 Accuracy: 38.04%
Epoch 3/70
Train Loss: 3.6973, Train Accuracy: 12.82%
Eval Loss: 3.4587, Eval Accuracy: 17.11%, Top-1 Accuracy: 17.11%, Top-5 Accuracy: 42.63%
Epoch 4/70
Train Loss: 3.5399, Train Accuracy: 15.41%
Eval Loss: 3.3352, Eval Accuracy: 18.56%, Top-1 Accuracy: 18.56%, Top-5 Accuracy: 46.31%
Epoch 5/70
Train Loss: 3.4200, Train Accuracy: 17.46%
Eval Loss: 3.2451, Eval Accuracy: 21.29%, Top-1 Accuracy: 21.29%, Top-5 Accuracy: 49.03%
Epoch 6/70
Train Loss: 3.3220, Train Accuracy: 19.00%
Eval Loss: 3.1867, Eval Accuracy: 21.75%, Top-1 Accuracy: 21.75%, Top-5 Accuracy: 50.09%
Epoch 7/70
Train Loss: 3.2362, Train Accuracy: 20.82%
Eval Loss: 3.1056, Eval Accuracy: 23.55%, Top-1 Accuracy: 23.55%, Top-5 Accuracy: 52.54%
Epo